# ch06 — 机器学习建模

本 notebook 演示：
1. LightGBM + OOF 交叉验证
2. Optuna 超参数搜索
3. SHAP 特征解释 & 理由码生成
4. Stacking 集成（LR + LightGBM 两层）
5. Rank-based Blending
6. 评分卡 vs LightGBM 综合对比

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')

from creditrisk.data import TARGET
from creditrisk.models import LGBMWithOOF
from creditrisk.ensemble import rank_blend, prob_blend, StackingEnsemble, pseudo_label
from creditrisk.evaluation import evaluate, compare_models
from creditrisk.utils import plot_roc_pr, plot_score_dist, plot_feature_importance, Timer

X_train = pd.read_parquet('../data/processed/X_train_features.parquet')
X_test  = pd.read_parquet('../data/processed/X_test_features.parquet')
y_train = pd.read_parquet('../data/processed/y_train.parquet')[TARGET]

print(f'X_train: {X_train.shape}  bad_rate: {y_train.mean():.4%}')

## 1. Baseline LightGBM（默认参数）

In [ ]:
baseline_params = {
    'num_leaves':        31,
    'min_child_samples': 100,
    'learning_rate':     0.05,
    'n_estimators':      2000,
    'feature_fraction':  0.8,
    'bagging_fraction':  0.8,
    'bagging_freq':      5,
    'lambda_l1':         0.1,
    'lambda_l2':         0.1,
}

with Timer('LightGBM Baseline（5-fold OOF）'):
    lgbm_v1 = LGBMWithOOF(n_splits=5, params=baseline_params)
    oof_v1, test_pred_v1 = lgbm_v1.fit_predict(X_train, y_train, X_test)

metrics_v1 = evaluate(y_train.values, oof_v1, label='LightGBM Baseline')

## 2. Optuna 超参数搜索

> 💡 完整搜索（100 trials）耗时较长，快速演示可设置 `n_trials=20`。

In [ ]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    params = {
        'objective':         'binary',
        'metric':            'auc',
        'verbosity':         -1,
        'num_leaves':        trial.suggest_int('num_leaves', 16, 128),
        'min_child_samples': trial.suggest_int('min_child_samples', 50, 500),
        'learning_rate':     trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'feature_fraction':  trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction':  trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq':      5,
        'lambda_l1':         trial.suggest_float('lambda_l1', 1e-3, 10.0, log=True),
        'lambda_l2':         trial.suggest_float('lambda_l2', 1e-3, 10.0, log=True),
        'scale_pos_weight':  (y_train==0).sum() / (y_train==1).sum(),
    }
    kf   = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    aucs = []
    for tr_idx, val_idx in kf.split(X_train, y_train):
        dtrain = lgb.Dataset(X_train.iloc[tr_idx], label=y_train.iloc[tr_idx])
        dval   = lgb.Dataset(X_train.iloc[val_idx], label=y_train.iloc[val_idx])
        m = lgb.train(params, dtrain, num_boost_round=500,
                      valid_sets=[dval],
                      callbacks=[lgb.early_stopping(30, verbose=False),
                                 lgb.log_evaluation(-1)])
        pred = m.predict(X_train.iloc[val_idx])
        aucs.append(roc_auc_score(y_train.iloc[val_idx], pred))
    return np.mean(aucs)

N_TRIALS = 30  # 快速演示
with Timer(f'Optuna 超参搜索（{N_TRIALS} trials）'):
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=N_TRIALS)

best_params = study.best_params
print(f'\n最优参数 (AUC={study.best_value:.4f})：')
print(best_params)

In [ ]:
# 用最优参数重训
tuned_params = {
    **best_params,
    'n_estimators': 2000,
}

with Timer('LightGBM Tuned（5-fold OOF）'):
    lgbm_v2 = LGBMWithOOF(n_splits=5, params=tuned_params)
    oof_v2, test_pred_v2 = lgbm_v2.fit_predict(X_train, y_train, X_test)

metrics_v2 = evaluate(y_train.values, oof_v2, label='LightGBM Tuned')

## 3. SHAP 特征解释

In [ ]:
import shap

# 用第一个 fold 模型做解释（代表性）
explainer   = shap.TreeExplainer(lgbm_v2.models_[0])
sample_size = min(5000, len(X_train))  # 取子样本加速
sample_idx  = np.random.choice(len(X_train), sample_size, replace=False)
X_sample    = X_train.iloc[sample_idx]

shap_values = explainer.shap_values(X_sample)
if isinstance(shap_values, list):
    shap_values = shap_values[1]  # 取违约类

# Beeswarm 图（全局特征重要性）
shap.summary_plot(shap_values, X_sample, max_display=20, plot_type='dot')

In [ ]:
# 单个申请人的解释（瀑布图）—— 用于生成拒绝理由码
idx = 0   # 选第 1 个样本
shap.waterfall_plot(
    shap.Explanation(
        values      = shap_values[idx],
        base_values = explainer.expected_value,
        data        = X_sample.iloc[idx],
        feature_names = X_sample.columns.tolist()
    )
)

In [ ]:
# 拒绝理由码生成示例
REASON_MAP = {
    'interestRate':    ('R01', '利率较高，反映贷款风险水平偏高'),
    'dti':             ('R02', '负债收入比偏高'),
    'loanToIncome':    ('R03', '申请金额与收入比例过高'),
    'ficoMean':        ('R04', 'FICO 信用分数偏低'),
    'revolUtil_norm':  ('R05', '循环信用利用率偏高'),
    'n_null_cnt':      ('R06', '关键信息字段缺失较多'),
}

def get_reject_reasons(shap_vals, feature_names, reason_map, top_n=3):
    reasons = []
    sorted_idx = np.argsort(shap_vals)[::-1]
    for i in sorted_idx:
        feat = feature_names[i]
        if feat in reason_map and shap_vals[i] > 0:
            code, desc = reason_map[feat]
            reasons.append({'code': code, 'desc': desc, 'impact': round(shap_vals[i], 4)})
        if len(reasons) >= top_n:
            break
    return reasons

reasons = get_reject_reasons(shap_values[idx], X_sample.columns.tolist(), REASON_MAP)
print('\n拒绝理由：')
for r in reasons:
    print(f"  [{r['code']}] {r['desc']}  (impact={r['impact']})")

## 4. 模型融合

In [ ]:
# Rank-based Blending（两个 LightGBM 版本）
blended_oof  = rank_blend([oof_v1, oof_v2], weights=[1, 2])
blended_test = rank_blend([test_pred_v1, test_pred_v2], weights=[1, 2])

metrics_blend = evaluate(y_train.values, blended_oof, label='Rank Blend (v1+v2)')

# 汇总对比
compare_models([metrics_v1, metrics_v2, metrics_blend])

## 5. Pseudo-labeling

用高置信度测试集样本扩充训练数据（信贷中等价于拒绝推断）。

In [ ]:
from creditrisk.ensemble import pseudo_label

# 用当前最优模型生成伪标签
X_pl, y_pl = pseudo_label(
    lgbm_v2,
    X_train, y_train, X_test,
    confidence_threshold=0.03,    # 极低/极高风险段
    bad_multiplier=2.0,
)
print(f'扩充后训练集：{len(X_pl):,} 条（原 {len(X_train):,} 条）')

# 在扩充数据上重训（快速演示用较少轮数）
pl_params = {**tuned_params, 'n_estimators': 1000}
lgbm_pl = LGBMWithOOF(n_splits=5, params=pl_params)
oof_pl, test_pl = lgbm_pl.fit_predict(X_pl, y_pl, X_test)

# 注意：伪标签模型的 OOF 包含伪标签行，只取原训练集部分评估
metrics_pl = evaluate(y_pl.values[:len(y_train)], oof_pl[:len(y_train)],
                      label='LightGBM + Pseudo-label')

## 6. 保存最终预测结果

In [ ]:
submission = pd.DataFrame({
    'id':        range(len(blended_test)),
    'isDefault': blended_test,
})
submission.to_csv('../data/processed/submission.csv', index=False)
print('已保存：../data/processed/submission.csv')
submission.head()